## Import Libraries and Load Dataset

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns

from whatthelang import WhatTheLang
import swifter as swf

%matplotlib inline

ModuleNotFoundError: No module named 'whatthelang'

In [13]:
!pip install future 
!pip install Cython 
!pip install cysignals 
!pip install pyfasttext 

  Using cached future-0.18.2.tar.gz (829 kB)
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=bbb5048b46c9773d2f1c68bb7585a9541e1cc3e6d7f88952f1845b3b1c098eb2
  Stored in directory: /Users/kode/Library/Caches/pip/wheels/2f/a0/d3/4030d9f80e6b3be787f19fc911b8e7aa462986a40ab1e4bb94
Successfully built future
  Using cached Cython-0.29.24-cp39-cp39-macosx_10_9_x86_64.whl (1.9 MB)
  Using cached cysignals-1.10.3.tar.gz (455 kB)
  Created wheel for cysignals: filename=cysignals-1.10.3-cp39-cp39-macosx_10_9_x86_64.whl size=232583 sha256=5e6ae0b9e2c2d9942d06a583c5acfe6d0e72bb1cb411d979df33debdb3401c82
  Stored in directory: /Users/kode/Library/Caches/pip/wheels/ad/04/20/a429b9738957c356da8626021df60f9532c7b622a7a72ed96d
Successfully built cysignals
  Using cached pyfasttext-0.4.6.tar.gz (244 kB)
  ERROR: Command errored out with exit status 1:
   command: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -u -c 'import io, os, sys, setuptools, 

Failed to build pyfasttext
    Running setup.py install for pyfasttext ... error
    ERROR: Command errored out with exit status 1:
     command: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/hp/y_p2cf_d4575l24bws2nk5fh0000gn/T/pip-install-9te4_b3z/pyfasttext_fd395458f2f34b8cba7594ffe52cde08/setup.py'"'"'; __file__='"'"'/private/var/folders/hp/y_p2cf_d4575l24bws2nk5fh0000gn/T/pip-install-9te4_b3z/pyfasttext_fd395458f2f34b8cba7594ffe52cde08/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /private/var/folders/hp/y_p2cf_d4575l24bws2nk5fh0000gn/T/pip-record-b7a4krpp/install-record.txt --single-version-externally-managed --compile --install-headers

In [2]:
%%time
df = pd.read_csv('./data/yelp_academic_base_dataset.csv')

CPU times: user 39.3 s, sys: 4.6 s, total: 43.9 s
Wall time: 44.4 s


In [3]:
df.head()

,businessId,meanStars,reviewCount,reviewStars,text,date
0,6iYb2HFDywm3zjuRg0shjw,4.0,86,5.0,Stopped in on a busy Friday night. Despite the...,2018-03-04 00:59:21
1,tCbdrRPZA0oiIYSmHG3J0w,4.0,126,4.0,Elephant's contacted me the same day I posted ...,2012-07-16 05:04:05
2,tCbdrRPZA0oiIYSmHG3J0w,4.0,126,5.0,I'm not usually a fan of airport food. I usual...,2015-04-28 21:11:10
3,tCbdrRPZA0oiIYSmHG3J0w,4.0,126,4.0,"If one must have breakfast at the airport, per...",2015-11-18 18:50:05
4,tCbdrRPZA0oiIYSmHG3J0w,4.0,126,5.0,"Reasonably priced, tasty local joint. Lots of ...",2011-11-30 20:15:41


## Data Cleaning

In [4]:
# Removing rows with missing values
print(f'Removing of {df.isna().values.sum()} values and respective rows...')
df.dropna(inplace=True)

Removing of 2 values and respective rows...


In [5]:
df.isna().values.any()

False

In [6]:
df.count()

businessId     6968547
meanStars      6968547
reviewCount    6968547
reviewStars    6968547
text           6968547
date           6968547
dtype: int64

In [7]:
# removing old rows
df = df[df['date'] > '2016-01-01']
df.count()

businessId     4215151
meanStars      4215151
reviewCount    4215151
reviewStars    4215151
text           4215151
date           4215151
dtype: int64

In [8]:
# balancing positive and negative reviews
negative = df[df['reviewStars'] <= 3]['text'].count()
positive = df[df['reviewStars'] > 3]['text'].count()
print(f'Positive reviews: {positive}')
print(f'Negative reviews: {negative}')
print('--------------------------')

# Splitting positive and negative dataframe
df_pos = df[df['reviewStars'] > 3]
df_neg = df[df['reviewStars'] <= 3]
print(f'Counting: {df_pos.size}, {df_neg.size}, {df.size}')

if positive > negative:
    df_pos = df_pos.iloc[:-(positive - negative)]
else:
    df_neg = df_neg.iloc[:-(negative - positive)]

df_pos = df_pos[:500000]
df_neg = df_neg[:500000]
# df_pos and df_neg has the same number of reviews
print(f'After balance phase :-> Positive: {df_pos.text.count()}, Negative: {df_pos.text.count()}')
print('Unify datasets')
df= pd.concat([df_pos,df_neg], axis=0)
df.head()

Positive reviews: 2845008
Negative reviews: 1370143
--------------------------
Counting: 17070048, 8220858, 25290906
After balance phase :-> Positive: 500000, Negative: 500000
Unify datasets


,businessId,meanStars,reviewCount,reviewStars,text,date
0,6iYb2HFDywm3zjuRg0shjw,4.0,86,5.0,Stopped in on a busy Friday night. Despite the...,2018-03-04 00:59:21
5,bvN78flM8NLprQ1a1y5dRg,4.5,13,4.0,Quaint little store with tons of amazing items...,2016-08-25 16:52:19
7,PE9uqAjdw0E4-8mjGl3wVA,4.0,14,5.0,I came in as a drop in. Casey Castleberry is ...,2016-08-12 23:38:16
10,D4JtQNTI4X3KcbzacDJsMw,3.5,169,4.0,"Good food, and good sized portions for the pri...",2016-05-19 06:56:54
11,HPA_qyMEddpAEtFof02ixg,4.0,39,4.0,Pizza here is delicious. I tried it once a few...,2016-03-07 17:58:53


In [ ]:
%%time
language = [detect_langs(i) for i in df_pos.text]
languages = [str(i[0]).split(' ')[0] for i in language]
print(f'{languages}')
df['language'] = languages

In [ ]:
# Removing not-english rows